In [1]:
import pandas as pd
import numpy as np

In [2]:
#list of all different tables 
data_type = ["stats","shooting","passing","passing_types","gca","defense","possession","playingtime"]

#iterate through all data types 
dataframes = {}

for i in data_type:
#initialize all the tables by loading the CSV files
    dataframes[f'{i}']= pd.read_csv(f'Big-5-European-Leagues-Stats_{i}_stats.csv')



In [3]:
#initialize all data sets 
shooting_df = dataframes['shooting']
passing_df = dataframes['passing']
passtype_df = dataframes['passing_types'] 
gsc_df = dataframes['gca']  #gsc stands for goal scoring creation
defense_df = dataframes['defense']
possession_df = dataframes['possession']
playingtime_df = dataframes['playingtime']

In [4]:
# Merge DataFrames with unique suffixes to avoid conflicts on duplicate columns
Big_5_master_df = shooting_df.merge(passing_df, on='Player', how='outer', suffixes=('', '_passing')) \
                       .merge(passtype_df, on='Player', how='outer', suffixes=('', '_passtype')) \
                       .merge(gsc_df, on='Player', how='outer', suffixes=('', '_gsc')) \
                       .merge(defense_df, on='Player', how='outer', suffixes=('', '_defense')) \
                       .merge(possession_df, on='Player', how='outer', suffixes=('', '_possession')) \
                       .merge(playingtime_df, on='Player', how='outer', suffixes=('', '_playingtime'))

print(Big_5_master_df.head())


       Rk                 Player   Nation    Pos      Squad  \
0     NaN  Aaron Ciammaglichella      NaN    NaN        NaN   
1   602.0        Aaron Cresswell  eng ENG  DF,FW   West Ham   
2  1136.0           Aaron Hickey  sct SCO     DF  Brentford   
3  1585.0          Aaron Malouda   fr FRA     FW      Lille   
4  2185.0         Aaron Ramsdale  eng ENG     GK    Arsenal   

                 Comp   Age    Born  90s  Gls  ...  onGA  +/-  +/-90  On-Off  \
0                 NaN   NaN     NaN  NaN  NaN  ...   NaN  NaN    NaN     NaN   
1  eng Premier League  33.0  1989.0  4.8  0.0  ...  11.0 -5.0  -1.03   -0.76   
2  eng Premier League  21.0  2002.0  7.9  0.0  ...  12.0 -2.0  -0.25   -0.02   
3          fr Ligue 1  17.0  2005.0  0.0  0.0  ...   0.0  0.0   0.00   -0.53   
4  eng Premier League  25.0  1998.0  6.0  0.0  ...   5.0  6.0   1.00   -0.75   

   onxG  onxGA  xG+/-  xG+/-90  On-Off.1  Matches_playingtime  
0   NaN    NaN    NaN      NaN       NaN              Matches  
1   6.6    5

In [9]:
#Create CSV for backup
Big_5_master_df.to_csv('Big_5_master.csv')

In [14]:
# Load the CSV files from transfrmarkt 
valuations_df = pd.read_csv('player_valuations.csv') #contains market value
players_df = pd.read_csv('players.csv') # contains player names 

# Convert 'date' column to datetime and filter for rows where the year is 2024
valuations_df['date'] = pd.to_datetime(valuations_df['date'], errors='coerce')
valuations_2024 = valuations_df[valuations_df['date'].dt.year == 2024]


# Sort by 'date' in descending order and drop duplicates to keep the most recent entry per player
valuations_2024 = valuations_2024.sort_values(by='date', ascending=False).drop_duplicates(subset='player_id', keep='first')

# Filter players for the last season being 2024
players_2024 = players_df[players_df['last_season'] == 2024]

# Select only relevant columns
valuations_2024 = valuations_2024[['player_id', 'market_value_in_eur']]
players_2024 = players_2024[['player_id', 'name']]

# Merge on 'player_id' column to get the final DataFrame
market_value_df = players_2024.merge(valuations_2024, on='player_id', how='inner')

# Display the merged DataFrame
print(market_value_df.head(10))

# Save the merged DataFrame to a CSV file
market_value_df.to_csv('market_value.csv', index=False)

   player_id                 name  market_value_in_eur
0       3333         James Milner              1000000
1       5336  Anastasios Tsokanis               350000
2       7161        Jonas Hofmann              7000000
3       7825           Pepe Reina               600000
4      11530        Lionel Carole               550000
5      12029      Ludovic Butelle               100000
6      12282          Daley Blind              3000000
7      12589         Craig Gordon               150000
8      12880   Dimitrios Sotiriou               100000
9      12907       Alessio Cragno              1500000


In [16]:

# Merge the master dataset with the market value dataset on 'Player' (master_df) and 'name' (market_value_df)
merged_master_df = Big_5_master_df.merge(market_value_df, left_on='Player', right_on='name', how='left')

# Drop the redundant 'name' column from the merged dataset if desired
merged_Big_5_master_df = merged_master_df.drop(columns=['name'])

# Display the first few rows of the merged DataFrame
print(merged_Big_5_master_df.head(10))

#save the merged DataFrame to a CSV file
merged_Big_5_master_df.to_csv('Big_5_with_market_value.csv', index=False)


       Rk                 Player   Nation    Pos           Squad  \
0     NaN  Aaron Ciammaglichella      NaN    NaN             NaN   
1   602.0        Aaron Cresswell  eng ENG  DF,FW        West Ham   
2  1136.0           Aaron Hickey  sct SCO     DF       Brentford   
3  1585.0          Aaron Malouda   fr FRA     FW           Lille   
4  2185.0         Aaron Ramsdale  eng ENG     GK         Arsenal   
5  2186.0           Aaron Ramsey  eng ENG  MF,FW         Burnley   
6  2415.0           Aaron Seydel   de GER     FW    Darmstadt 98   
7  2741.0      Aaron Wan-Bissaka  eng ENG     DF  Manchester Utd   
8   823.0        Aarón Escandell   es ESP     GK      Las Palmas   
9  1630.0           Aarón Martín   es ESP     DF           Genoa   

                 Comp   Age    Born   90s  Gls  ...  +/-90  On-Off  onxG  \
0                 NaN   NaN     NaN   NaN  NaN  ...    NaN     NaN   NaN   
1  eng Premier League  33.0  1989.0   4.8  0.0  ...  -1.03   -0.76   6.6   
2  eng Premier League  